Decorators are a very powerful and useful tool in Python since it allows programmers to modify the behaviour of a function or class. Decorators allow us to wrap another function in order to extend the behaviour of the wrapped function, without permanently modifying it. But before diving deep into decorators let us understand some concepts that will come in handy in learning the decorators.

In [2]:
'''
In This function we returning function under function
'''

def function1(num):
    if num==0:
        return print
    if num==1:
        return int
a = function1(1)

print(a)
    


<class 'int'>


In [9]:
def decoratorfxn(func1):
    def nowexec():
        print("Execution now")
        func1() # another function executed inside another function
        print("Executed")
    return nowexec

def who_is_penguin():
    print("Penguin lives in extream cold regions")
    
who_is_penguin = decoratorfxn(who_is_penguin)  
who_is_penguin() # calling
# function who_is_penguin is passed as argument to decorator function


        
        
        

Execution now
Penguin lives in extream cold regions
Executed


Writing decorator with @ 

In [23]:
def decorator(func1):
    def nowexec():
        print("Execution now")
      
        func1() # another function executed inside another function
        print("Executed")
    return nowexec

@decorator
def who_is_penguin():    
    print("Penguin lives in extream cold regions")
    
    
who_is_penguin()
# function who_is_penguin is passed as argument to dec1 function


        
        

Execution now
Penguin lives in extream cold regions
Executed


In [24]:
print(who_is_penguin.__name__)
print(who_is_penguin.__doc__)


nowexec
None


*Decorator Application (Logger , Starcked Decorators)*

In [25]:
#Decorartor function
def logged(fn):
    from functools import wraps
    from datetime import datetime, timezone
    
    @wraps(fn)
    def inner(*args, **kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        print('{0}: called {1}'.format(fn.__name__, run_dt))
        return result
        
    return inner

In [26]:
@logged
def func_1():
    pass

In [28]:
func_1()

func_1: called 2023-03-05 08:17:00.343070+00:00


In [29]:
#Decorator Function
def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('{0} ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result
    
    return inner

In [36]:
@timed #factorial function get passed to timed function
@logged #factorial function get passed to  function

def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

In [38]:
factorial(10)

factorial: called 2023-03-05 08:27:11.357660+00:00
factorial ran for 0.000498s


3628800

 two very simple decorators

In [39]:
def dec_1(fn):
    def inner():
        print('running dec_1')
        return fn()
    return inner

In [40]:
def dec_2(fn):
    def inner():
        print('running dec_2')
        return fn()
    return inner

In [41]:
@dec_1
@dec_2
def my_func():
    print('running my_func')

In [43]:
my_func()

running dec_1
running dec_2
running my_func


In [44]:
@dec_2
@dec_1
def my_func():
    print('running my_func')

In [45]:
my_func()

running dec_2
running dec_1
running my_func


You may wonder whether this really matters in practice. And yes, it can.

Consider an API that contains various functions that can be called. However, endpoints are secured and can only be run by authenticated users who have some specific role(s). If they do not have the role you want to return an unauthorized error. But if they do, then you want to log that they called the endpoint.

In this case you may have one decorator that is used to check authentication and permissions (and immediately return an unauthorized error from the API if applicable), and the other to log the call.

If you decorated it this way:

@log
@authorize
def my_endpoint():
    pass
then the call would always be logged.

But, in this instance:

@authorize
@log
def my_endpoint():
    pass
your endpoint would only get logged if the user passed the authorize test.

### Decorators Application (Memoization)

In [48]:
#simple program
def fib(n):
    print ('Calculating fib({0})'.format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

fib(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)
Calculating fib(2)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


5

'''
It would be better if we could somehow "store" these results, so if we have calculated fib(4) and fib(3) before, we could simply recall the these values when calculating fib(5) = fib(4) + fib(3) instead of recalculating them.

This concept of improving the efficiency of our code by caching pre-calculated values so they do not need to be re-calcualted every time, is called "memoization"

We can approach this using a simple class and a dictionary that stores any Fibonacci number that's already been calculated:

'''

In [51]:

class Fib:
    def __init__(self):
        self.cache = {1: 1, 2: 1}
    
    def fib(self, n):
        if n not in self.cache:
            print('Calculating fib({0})'.format(n))
            self.cache[n] = self.fib(n-1) + self.fib(n-2)
        return self.cache[n]

In [52]:
f = Fib()
f.fib(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


5

In [53]:
#  using a closure:

def fib():
    cache = {1: 1, 2: 2}
    
    def calc_fib(n):
        if n not in cache:
            print('Calculating fib({0})'.format(n))
            cache[n] = calc_fib(n-1) + calc_fib(n-2)
        return cache[n]
    
    return calc_fib

In [54]:
f = fib()

In [55]:
f(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)


8

In [56]:
# using a decorator:

from functools import wraps

def memoize_fib(fn):
    cache = dict()
    
    @wraps(fn)
    def inner(n):
        if n not in cache:
            cache[n] = fn(n)
        return cache[n]
    
    return inner

In [57]:
@memoize_fib
def fib(n):
    print ('Calculating fib({0})'.format(n))
    return 1 if n < 3 else fib(n-1) + fib(n-2)

In [58]:
fib(5)

Calculating fib(5)
Calculating fib(4)
Calculating fib(3)
Calculating fib(2)
Calculating fib(1)


5

It's in the, you guessed it, functools module, and is called lru_cache and is going to be quite a bit more efficient compared to the rudimentary memoization example we did above.

[LRU Cache = Least Recently Used caching: since the cache is not unlimited, at some point cached entries need to be discarded, and the least recently used entries are discarded first]

In [59]:
from functools import lru_cache
@lru_cache()
def fact(n):
    print("Calculating fact({0})".format(n))
    return 1 if n < 2 else n * fact(n-1)

In [60]:
fact(5)

Calculating fact(5)
Calculating fact(4)
Calculating fact(3)
Calculating fact(2)
Calculating fact(1)


120